In [4]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [5]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [6]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [7]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [8]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [9]:
x=np.empty((12*471,9*18))
y=np.empty((12*471,1))

In [10]:
for m in range(12):
    for g in range(471):
        x[m*471+g:m*471+g+1,:]=year_data[m][:,g:g+9].reshape(1,-1)
        y[m*471+g:m*471+g+1,:]=year_data[m][9,g+9]
        

- just test no normalize

In [11]:
import torch 
from torch import optim,nn

In [12]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [13]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [14]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [15]:
x1=torch.Tensor(x)
y1=torch.Tensor(y)

In [13]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 157.039215 , test loss : 165.698273
epoch = 2 train_loss : 122.786354 , test loss : 133.699951
epoch = 3 train_loss : 97.969376 , test loss : 107.753044
epoch = 4 train_loss : 86.400795 , test loss : 95.250854
epoch = 5 train_loss : 80.206955 , test loss : 88.191490
epoch = 6 train_loss : 72.480095 , test loss : 80.046837
epoch = 7 train_loss : 70.876518 , test loss : 77.747261
epoch = 8 train_loss : 62.244366 , test loss : 69.324951
epoch = 9 train_loss : 59.752819 , test loss : 66.811470
epoch = 10 train_loss : 57.522305 , test loss : 64.175575
epoch = 11 train_loss : 54.193691 , test loss : 60.202473
epoch = 12 train_loss : 51.548103 , test loss : 57.405933
epoch = 13 train_loss : 49.598629 , test loss : 55.328747
epoch = 14 train_loss : 49.590546 , test loss : 55.273438
epoch = 15 train_loss : 46.332603 , test loss : 51.781460
epoch = 16 train_loss : 44.983685 , test loss : 50.217907
epoch = 17 train_loss : 44.171879 , test loss : 49.350605
epoch = 18 train_l

epoch = 16 train_loss : 44.798546 , test loss : 45.330383
epoch = 18 train_loss : 42.475723 , test loss : 42.940014
epoch = 20 train_loss : 41.252346 , test loss : 41.718384
epoch = 21 train_loss : 40.963718 , test loss : 41.705227
epoch = 22 train_loss : 39.661907 , test loss : 40.011219
epoch = 23 train_loss : 39.217190 , test loss : 39.396023
epoch = 24 train_loss : 38.960449 , test loss : 39.208260
epoch = 27 train_loss : 37.547924 , test loss : 37.930187
epoch = 29 train_loss : 36.987709 , test loss : 37.319504
epoch = 31 train_loss : 36.356220 , test loss : 37.187477
epoch = 37 train_loss : 35.495014 , test loss : 36.092602
epoch = 38 train_loss : 35.280697 , test loss : 36.027203
epoch = 44 train_loss : 34.856483 , test loss : 35.462978
epoch = 51 train_loss : 34.313686 , test loss : 35.245361
epoch = 60 train_loss : 33.693222 , test loss : 34.783596
epoch = 69 train_loss : 33.636047 , test loss : 34.728920
epoch = 82 train_loss : 33.511898 , test loss : 34.581509
epoch = 89 tra

In [17]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],512),nn.Linear(512,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 290.766388 , test loss : 269.512115
epoch = 2 train_loss : 103.084366 , test loss : 134.009445
epoch = 3 train_loss : 79.996941 , test loss : 101.958191
epoch = 4 train_loss : 70.435394 , test loss : 84.803787
epoch = 5 train_loss : 64.179726 , test loss : 74.874535
epoch = 6 train_loss : 59.422081 , test loss : 70.178734
epoch = 7 train_loss : 56.428925 , test loss : 67.311516
epoch = 8 train_loss : 54.308441 , test loss : 60.004467
epoch = 9 train_loss : 51.060757 , test loss : 57.502617
epoch = 10 train_loss : 50.022552 , test loss : 55.552086
epoch = 11 train_loss : 47.512028 , test loss : 53.037868
epoch = 12 train_loss : 46.209293 , test loss : 51.636555
epoch = 13 train_loss : 44.205700 , test loss : 51.578796
epoch = 15 train_loss : 45.089931 , test loss : 48.935425
epoch = 16 train_loss : 41.220982 , test loss : 47.141804
epoch = 20 train_loss : 38.471390 , test loss : 46.138603
epoch = 22 train_loss : 39.110271 , test loss : 44.030369
epoch = 24 train_l

epoch = 579 train_loss : 36.938469 , test loss : 21.137606
epoch = 842 train_loss : 36.743961 , test loss : 21.071686
epoch = 875 train_loss : 37.535267 , test loss : 20.861010
--------------------------------------------------------------------------
fold 4,train loss mean : 37.535267,test loss : 20.861010
-------------------------------------------------------------------------
epoch = 1 train_loss : 344.158264 , test loss : 364.149841
epoch = 2 train_loss : 98.619385 , test loss : 119.195351
epoch = 3 train_loss : 84.213676 , test loss : 94.371712
epoch = 4 train_loss : 73.286697 , test loss : 87.182747
epoch = 5 train_loss : 68.329010 , test loss : 76.775963
epoch = 6 train_loss : 62.501350 , test loss : 72.582199
epoch = 8 train_loss : 55.265423 , test loss : 63.680378
epoch = 9 train_loss : 56.973656 , test loss : 58.588451
epoch = 10 train_loss : 50.257824 , test loss : 55.361938
epoch = 12 train_loss : 46.380188 , test loss : 52.420948
epoch = 14 train_loss : 44.755901 , test l

### just test data col normalize

In [18]:
x.shape

(5652, 162)

In [16]:
x2=pd.DataFrame(x)
x2=x2.apply(lambda x:((x-x.mean())/x.std()))

In [17]:
x22=torch.Tensor(x2.values)
y22=torch.Tensor(y)

In [21]:
def get_net():
    return nn.Sequential(nn.Linear(x22.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x22,y22,256,0,0)

epoch = 1 train_loss : 563.576843 , test loss : 832.734558
epoch = 11 train_loss : 244.796875 , test loss : 767.408997
epoch = 12 train_loss : 214.190063 , test loss : 677.944275
epoch = 13 train_loss : 185.046494 , test loss : 552.179260
epoch = 14 train_loss : 155.866241 , test loss : 491.534088
epoch = 15 train_loss : 132.594147 , test loss : 383.075348
epoch = 16 train_loss : 107.768951 , test loss : 328.814972
epoch = 17 train_loss : 87.841248 , test loss : 224.141357
epoch = 18 train_loss : 71.900284 , test loss : 181.749496
epoch = 19 train_loss : 59.911762 , test loss : 129.574921
epoch = 20 train_loss : 51.394901 , test loss : 110.958374
epoch = 21 train_loss : 45.188522 , test loss : 86.687241
epoch = 22 train_loss : 41.066101 , test loss : 67.410683
epoch = 23 train_loss : 38.290638 , test loss : 59.066189
epoch = 24 train_loss : 37.328880 , test loss : 50.510220
epoch = 25 train_loss : 36.293625 , test loss : 48.489433
epoch = 26 train_loss : 35.523125 , test loss : 48.2970

epoch = 35 train_loss : 37.688641 , test loss : 23.398241
epoch = 39 train_loss : 37.174675 , test loss : 23.071039
epoch = 42 train_loss : 37.052330 , test loss : 22.730919
epoch = 47 train_loss : 36.647984 , test loss : 22.539539
epoch = 51 train_loss : 36.479469 , test loss : 22.492062
epoch = 72 train_loss : 35.717010 , test loss : 22.463718
epoch = 94 train_loss : 35.570087 , test loss : 22.332874
--------------------------------------------------------------------------
fold 4,train loss mean : 35.570087,test loss : 22.332874
-------------------------------------------------------------------------
epoch = 1 train_loss : 627.315674 , test loss : 616.007874
epoch = 2 train_loss : 572.924744 , test loss : 612.647827
epoch = 3 train_loss : 543.352234 , test loss : 608.106201
epoch = 4 train_loss : 514.450195 , test loss : 601.866272
epoch = 9 train_loss : 361.626282 , test loss : 592.471252
epoch = 10 train_loss : 326.927460 , test loss : 554.085449
epoch = 11 train_loss : 291.85397

In [22]:
def get_net():
    return nn.Sequential(nn.Linear(x22.shape[1],512),nn.Linear(512,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x22,y22,256,0,0)

epoch = 1 train_loss : 519.522644 , test loss : 856.301941
epoch = 6 train_loss : 273.289673 , test loss : 833.894287
epoch = 7 train_loss : 227.867020 , test loss : 637.556885
epoch = 8 train_loss : 166.867188 , test loss : 513.016724
epoch = 9 train_loss : 123.539467 , test loss : 349.019836
epoch = 10 train_loss : 83.953239 , test loss : 222.995239
epoch = 11 train_loss : 60.247231 , test loss : 139.517242
epoch = 12 train_loss : 45.431210 , test loss : 77.466156
epoch = 13 train_loss : 39.015503 , test loss : 56.396309
epoch = 14 train_loss : 36.864075 , test loss : 47.060932
epoch = 15 train_loss : 35.774174 , test loss : 44.592182
epoch = 16 train_loss : 34.431145 , test loss : 42.531151
epoch = 19 train_loss : 34.662807 , test loss : 40.981411
epoch = 24 train_loss : 33.019962 , test loss : 40.965160
epoch = 31 train_loss : 32.654114 , test loss : 40.774384
epoch = 33 train_loss : 32.224751 , test loss : 39.354095
epoch = 65 train_loss : 31.776569 , test loss : 38.947426
-------

--------------------------------------------------------------------------
fold 5,train loss mean : 33.054325,test loss : 31.552277
-------------------------------------------------------------------------
5 fold ,total train loss mean : 32.586014,total test loss mean : 34.955980 
-----------------------------------------------------------------------------


- just test linear model

In [18]:
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [19]:
n_folds=5

def rmlse_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x)
    rmse=np.sqrt(-cross_val_score(model,x,y,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [20]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
score=rmlse_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


Lasso score: 5.9364 (0.6516)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [21]:
np.sqrt(34.326048 )

5.8588435719005165

In [22]:
enet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9))
scoreo=rmlse_cv(enet)
print("\n enet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


 enet score: 5.9364 (0.6516)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [29]:
ridge=make_pipeline(RobustScaler(),Ridge(alpha=0.5))
score=rmlse_cv(ridge)
print("\n ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


 ridge score: 5.9404 (0.6500)



In [30]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)
score = rmlse_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Gradient Boosting score: 6.9629 (1.0416)



In [31]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmlse_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[10:52:21] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:52:32] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:52:41] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [33]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmlse_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

LGBM score: 6.6617 (0.9785)



### just 18 test features normalize

In [34]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data[i]))
all_train_data=pd.concat(month_list,axis=1).T

In [35]:
month_list[11].shape

(18, 480)

In [36]:
all_train_data.shape

(5760, 18)

- check reshape data is right ?

In [37]:
all_train_data.iloc[-5:,9:]

,9,10,11,12,13,14,15,16,17
475,37.0,0.0,76.0,2.6,1.9,109.0,97.0,1.0,1.4
476,28.0,0.0,80.0,2.2,1.9,108.0,107.0,1.7,1.3
477,17.0,0.0,82.0,2.3,1.9,114.0,118.0,1.5,1.6
478,24.0,0.0,84.0,2.3,2.0,108.0,100.0,2.0,1.8
479,29.0,0.0,84.0,2.3,2.0,109.0,105.0,2.0,2.0


In [38]:
all_train_data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.527413,1.702396,0.388363,0.140427,2.135729,10.125990,12.247726,31.905469,42.709201,21.414236,0.200625,73.229167,2.763125,1.839653,156.329271,158.482795,2.297240,1.712760
std,6.290152,0.125265,0.323573,0.104645,2.282155,6.187555,7.577133,18.703486,26.222292,16.662537,2.045443,13.361351,1.816940,0.181839,95.745881,94.697432,1.065408,1.062683
min,-12.300000,-0.200000,-0.120000,0.000000,-1.100000,0.000000,-2.400000,0.000000,0.000000,-1.000000,0.000000,29.000000,-1.600000,-0.200000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,29.250000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


### correct outlier value

In [39]:
year_data1=year_data.copy()

In [40]:
# temperature always >=0 in Taiwan and change fast when temperature <=0 ,so correct it
for idx in [0]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <=0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
---------- pm2.5 start : 
 -3.1 

-8.0
-7.2
-6.8
-6.5
-7.1
-7.4
-8.1
-8.3
-8.4
-9.3
-10.6
-11.2
-12.1
0.0
0.0
0.0
0.0
-12.3
pm2.5 end ------------
correct value between  : 19.0 18.0
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11


In [41]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [42]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.702396,0.388363,0.140427,2.135729,10.125990,12.247726,31.905469,42.709201,21.414236,0.200625,73.229167,2.763125,1.839653,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.125265,0.323573,0.104645,2.282155,6.187555,7.577133,18.703486,26.222292,16.662537,2.045443,13.361351,1.816940,0.181839,95.745881,94.697432,1.065408,1.062683
min,6.700000,-0.200000,-0.120000,0.000000,-1.100000,0.000000,-2.400000,0.000000,0.000000,-1.000000,0.000000,29.000000,-1.600000,-0.200000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,29.250000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [43]:
for idx in [1,2,4,6,9,12,13]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >=0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.2 

-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
pm2.5 end ------------
correct value between  : 1.8 1.8
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.12 

pm2.5 end ------------
correct value between  : 0.34 0.26
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 2.5 0.3
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 0.1 1.0
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -1.1 

pm2.5 end ------------
correct value between  : 6.6 1.2
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
---------

In [44]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [45]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.705521,0.388436,0.140427,2.136970,10.125990,12.254115,31.905469,42.709201,21.534201,0.200625,73.229167,2.769089,1.843012,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.100203,0.323505,0.104645,2.281611,6.187555,7.571422,18.703486,26.222292,16.576035,2.045443,13.361351,1.809743,0.163008,95.745881,94.697432,1.065408,1.062683
min,6.700000,0.000000,0.080000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [46]:
x3=np.empty((12*471,18*9))
y3=np.empty((12*471,1))

In [47]:
for m in range(12):
    for g in range(471):
        x3[m*471+g:m*471+g+1,:]=year_data1[m][:,g:g+9].reshape(1,-1)
        y3[m*471+g:m*471+g+1,:]=year_data1[m][9,g+9]

In [48]:
year_data[11][-5,:5]

array([1.7, 1.7, 1.7, 1.7, 1.8])

In [49]:
year_data1[11][-5,:5]

array([1.7, 1.7, 1.7, 1.7, 1.8])

In [50]:
x[-5,-5:]

array([1.3, 1.7, 0.7, 0.4, 1.1])

In [51]:
x3[-5,-5:]

array([1.3, 1.7, 0.7, 0.4, 1.1])

In [52]:
y[-5:]

array([[37.],
       [28.],
       [17.],
       [24.],
       [29.]])

In [53]:
y3[-5:]

array([[37.],
       [28.],
       [17.],
       [24.],
       [29.]])

In [54]:
y[y<0]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [55]:
y3[y3<0]

array([], dtype=float64)

In [56]:
x3[x3<0]

array([], dtype=float64)

In [57]:
x33=pd.DataFrame(x3)
x33=x33.apply(lambda x: (x-x.mean())/x.std())
x33=x33.values
y33=y3

In [58]:
x333=torch.Tensor(x33)
y333=torch.Tensor(y33)

In [52]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 150.251907 , test loss : 155.604050
epoch = 2 train_loss : 99.558205 , test loss : 104.195030
epoch = 3 train_loss : 80.369507 , test loss : 87.102425
epoch = 4 train_loss : 69.669907 , test loss : 75.825897
epoch = 5 train_loss : 64.667076 , test loss : 70.746811
epoch = 6 train_loss : 61.921131 , test loss : 67.884300
epoch = 7 train_loss : 56.259201 , test loss : 62.566200
epoch = 8 train_loss : 52.487587 , test loss : 58.312618
epoch = 9 train_loss : 50.237362 , test loss : 55.953819
epoch = 10 train_loss : 48.078827 , test loss : 53.591438
epoch = 12 train_loss : 45.296902 , test loss : 50.552940
epoch = 13 train_loss : 43.835632 , test loss : 48.931911
epoch = 14 train_loss : 43.257683 , test loss : 48.668621
epoch = 15 train_loss : 43.066242 , test loss : 48.321022
epoch = 16 train_loss : 41.003517 , test loss : 46.407482
epoch = 18 train_loss : 40.249680 , test loss : 45.904747
epoch = 19 train_loss : 38.598541 , test loss : 44.173035
epoch = 20 train_los

epoch = 47 train_loss : 34.565628 , test loss : 35.519215
epoch = 52 train_loss : 34.032391 , test loss : 35.292900
epoch = 55 train_loss : 34.050674 , test loss : 35.128277
epoch = 58 train_loss : 33.804146 , test loss : 34.602654
epoch = 87 train_loss : 33.378014 , test loss : 34.423214
epoch = 92 train_loss : 33.236534 , test loss : 34.405964
epoch = 114 train_loss : 32.970219 , test loss : 34.267426
epoch = 125 train_loss : 32.875366 , test loss : 34.185539
epoch = 126 train_loss : 32.862984 , test loss : 34.093227
epoch = 155 train_loss : 33.150898 , test loss : 33.923546
epoch = 206 train_loss : 32.652462 , test loss : 33.831314
epoch = 599 train_loss : 32.596390 , test loss : 33.752548
--------------------------------------------------------------------------
fold 4,train loss mean : 32.596390,test loss : 33.752548
-------------------------------------------------------------------------
epoch = 1 train_loss : 217.226624 , test loss : 210.688721
epoch = 2 train_loss : 124.173622

In [63]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],512),nn.Linear(512,256),nn.Linear(256,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 242.862030 , test loss : 430.585968
epoch = 2 train_loss : 119.124214 , test loss : 118.581306
epoch = 3 train_loss : 92.216858 , test loss : 113.473755
epoch = 4 train_loss : 79.558067 , test loss : 93.636253
epoch = 5 train_loss : 68.957764 , test loss : 77.322014
epoch = 6 train_loss : 62.653736 , test loss : 67.041039
epoch = 7 train_loss : 55.060841 , test loss : 60.360199
epoch = 8 train_loss : 51.390102 , test loss : 59.180382
epoch = 9 train_loss : 50.936089 , test loss : 54.854618
epoch = 10 train_loss : 43.929916 , test loss : 50.223682
epoch = 12 train_loss : 41.247108 , test loss : 49.901432
epoch = 13 train_loss : 39.674068 , test loss : 46.727482
epoch = 16 train_loss : 40.105263 , test loss : 46.483082
epoch = 17 train_loss : 38.295731 , test loss : 44.586624
epoch = 19 train_loss : 40.744301 , test loss : 44.009396
epoch = 26 train_loss : 34.560661 , test loss : 41.944489
epoch = 28 train_loss : 34.315250 , test loss : 41.782665
epoch = 35 train_l

epoch = 49 train_loss : 34.928020 , test loss : 33.869534
epoch = 51 train_loss : 36.700760 , test loss : 33.787556
epoch = 63 train_loss : 35.367897 , test loss : 33.556740
epoch = 64 train_loss : 34.876911 , test loss : 32.767750
epoch = 76 train_loss : 34.153156 , test loss : 32.471138
epoch = 137 train_loss : 33.618958 , test loss : 32.308193
epoch = 166 train_loss : 34.511906 , test loss : 32.157074
epoch = 198 train_loss : 33.693707 , test loss : 31.952950
epoch = 460 train_loss : 33.335690 , test loss : 31.911142
epoch = 670 train_loss : 33.825115 , test loss : 31.892073
epoch = 784 train_loss : 33.554974 , test loss : 31.700590
--------------------------------------------------------------------------
fold 5,train loss mean : 33.554974,test loss : 31.700590
-------------------------------------------------------------------------
5 fold ,total train loss mean : 33.122111,total test loss mean : 34.370378 
--------------------------------------------------------------------------

In [105]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],8),nn.Linear(8,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 288.760529 , test loss : 299.105316
epoch = 2 train_loss : 234.510315 , test loss : 249.888062
epoch = 3 train_loss : 229.500900 , test loss : 245.100098
epoch = 4 train_loss : 221.382858 , test loss : 236.312546
epoch = 5 train_loss : 214.687164 , test loss : 229.509644
epoch = 6 train_loss : 208.009369 , test loss : 222.841476
epoch = 7 train_loss : 201.419632 , test loss : 216.145477
epoch = 8 train_loss : 195.092239 , test loss : 209.845413
epoch = 9 train_loss : 188.901154 , test loss : 203.525360
epoch = 10 train_loss : 183.050842 , test loss : 197.447571
epoch = 11 train_loss : 177.628250 , test loss : 191.980637
epoch = 12 train_loss : 172.545654 , test loss : 186.748230
epoch = 13 train_loss : 167.733047 , test loss : 181.838089
epoch = 14 train_loss : 163.276962 , test loss : 177.231750
epoch = 15 train_loss : 159.083923 , test loss : 172.846954
epoch = 16 train_loss : 155.173172 , test loss : 168.875824
epoch = 17 train_loss : 151.477036 , test loss : 

epoch = 150 train_loss : 53.291855 , test loss : 59.281574
epoch = 151 train_loss : 53.142475 , test loss : 59.100426
epoch = 152 train_loss : 52.649277 , test loss : 58.768467
epoch = 154 train_loss : 52.350552 , test loss : 58.456951
epoch = 155 train_loss : 52.252956 , test loss : 58.362350
epoch = 156 train_loss : 52.162903 , test loss : 58.284046
epoch = 157 train_loss : 51.896244 , test loss : 57.977318
epoch = 158 train_loss : 51.652313 , test loss : 57.675774
epoch = 159 train_loss : 51.408676 , test loss : 57.392944
epoch = 160 train_loss : 51.537674 , test loss : 57.357029
epoch = 161 train_loss : 51.573090 , test loss : 57.348331
epoch = 162 train_loss : 51.365749 , test loss : 57.123726
epoch = 163 train_loss : 50.794296 , test loss : 56.622768
epoch = 164 train_loss : 50.613033 , test loss : 56.541996
epoch = 165 train_loss : 50.449718 , test loss : 56.373264
epoch = 167 train_loss : 50.368111 , test loss : 56.317455
epoch = 168 train_loss : 50.050758 , test loss : 55.9693

epoch = 372 train_loss : 35.313892 , test loss : 41.124584
epoch = 374 train_loss : 35.264469 , test loss : 41.046535
epoch = 378 train_loss : 35.344063 , test loss : 41.030548
epoch = 380 train_loss : 35.121494 , test loss : 40.943882
epoch = 381 train_loss : 35.099327 , test loss : 40.893944
epoch = 384 train_loss : 35.084538 , test loss : 40.880508
epoch = 385 train_loss : 35.015724 , test loss : 40.830517
epoch = 390 train_loss : 34.911461 , test loss : 40.754025
epoch = 393 train_loss : 34.921089 , test loss : 40.717049
epoch = 394 train_loss : 34.929749 , test loss : 40.702599
epoch = 398 train_loss : 34.828613 , test loss : 40.648743
epoch = 399 train_loss : 34.788113 , test loss : 40.614304
epoch = 402 train_loss : 34.666615 , test loss : 40.573494
epoch = 403 train_loss : 34.759628 , test loss : 40.570953
epoch = 407 train_loss : 34.578041 , test loss : 40.525467
epoch = 410 train_loss : 34.514271 , test loss : 40.450233
epoch = 412 train_loss : 34.477901 , test loss : 40.4234

epoch = 33 train_loss : 132.719315 , test loss : 150.769745
epoch = 34 train_loss : 130.729111 , test loss : 148.753815
epoch = 35 train_loss : 128.893021 , test loss : 146.882385
epoch = 36 train_loss : 127.122955 , test loss : 145.417603
epoch = 37 train_loss : 125.385361 , test loss : 143.578232
epoch = 38 train_loss : 123.784668 , test loss : 141.973450
epoch = 39 train_loss : 122.241714 , test loss : 140.430420
epoch = 40 train_loss : 120.738068 , test loss : 138.911957
epoch = 41 train_loss : 119.343369 , test loss : 137.266495
epoch = 42 train_loss : 117.946274 , test loss : 135.876160
epoch = 43 train_loss : 116.632790 , test loss : 134.478088
epoch = 44 train_loss : 115.374565 , test loss : 133.228760
epoch = 45 train_loss : 114.149895 , test loss : 132.010483
epoch = 46 train_loss : 112.949539 , test loss : 130.518661
epoch = 47 train_loss : 111.775475 , test loss : 129.395218
epoch = 48 train_loss : 110.656464 , test loss : 128.275238
epoch = 49 train_loss : 109.603485 , tes

epoch = 190 train_loss : 52.563210 , test loss : 63.858692
epoch = 191 train_loss : 52.012016 , test loss : 63.544781
epoch = 192 train_loss : 51.908978 , test loss : 63.377678
epoch = 193 train_loss : 51.687553 , test loss : 63.199482
epoch = 194 train_loss : 51.647705 , test loss : 63.124939
epoch = 195 train_loss : 51.241505 , test loss : 62.935001
epoch = 196 train_loss : 51.204723 , test loss : 62.718201
epoch = 200 train_loss : 50.391014 , test loss : 62.119194
epoch = 202 train_loss : 50.053406 , test loss : 61.835754
epoch = 203 train_loss : 49.898346 , test loss : 61.615543
epoch = 204 train_loss : 49.752811 , test loss : 61.579170
epoch = 205 train_loss : 49.604626 , test loss : 61.341122
epoch = 206 train_loss : 49.455776 , test loss : 61.180130
epoch = 207 train_loss : 49.252289 , test loss : 61.040607
epoch = 209 train_loss : 48.973236 , test loss : 60.742294
epoch = 212 train_loss : 48.471134 , test loss : 60.406094
epoch = 213 train_loss : 48.356258 , test loss : 60.1751

...

epoch = 487 train_loss : 35.597206 , test loss : 36.901684
epoch = 490 train_loss : 35.576408 , test loss : 36.825768
epoch = 497 train_loss : 35.461536 , test loss : 36.748646
epoch = 502 train_loss : 35.384346 , test loss : 36.659729
epoch = 505 train_loss : 35.301750 , test loss : 36.654308
epoch = 510 train_loss : 35.232712 , test loss : 36.568283
epoch = 516 train_loss : 35.179352 , test loss : 36.493225
epoch = 518 train_loss : 35.110737 , test loss : 36.465855
epoch = 520 train_loss : 35.109180 , test loss : 36.414772
epoch = 527 train_loss : 34.985950 , test loss : 36.382778
epoch = 530 train_loss : 35.096497 , test loss : 36.330425
epoch = 535 train_loss : 34.882980 , test loss : 36.274750
epoch = 539 train_loss : 34.835377 , test loss : 36.255936
epoch = 540 train_loss : 34.821186 , test loss : 36.238926
epoch = 541 train_loss : 34.932709 , test loss : 36.196182
epoch = 543 train_loss : 34.821442 , test loss : 36.136360
epoch = 548 train_loss : 34.744690 , test loss : 36.1308

epoch = 5 train_loss : 396.441620 , test loss : 404.982910
epoch = 6 train_loss : 376.913208 , test loss : 383.944366
epoch = 7 train_loss : 358.417664 , test loss : 363.824524
epoch = 8 train_loss : 340.220520 , test loss : 344.203705
epoch = 9 train_loss : 323.078064 , test loss : 325.789948
epoch = 10 train_loss : 306.856445 , test loss : 308.005585
epoch = 11 train_loss : 291.486450 , test loss : 291.461334
epoch = 12 train_loss : 276.987213 , test loss : 275.986847
epoch = 13 train_loss : 263.574585 , test loss : 261.539612
epoch = 14 train_loss : 251.216446 , test loss : 248.182434
epoch = 15 train_loss : 239.573395 , test loss : 235.837234
epoch = 16 train_loss : 228.946091 , test loss : 224.446442
epoch = 17 train_loss : 218.886887 , test loss : 213.751282
epoch = 18 train_loss : 209.740936 , test loss : 204.051254
epoch = 19 train_loss : 201.189774 , test loss : 194.963959
epoch = 20 train_loss : 193.402969 , test loss : 186.712082
epoch = 21 train_loss : 186.186508 , test los

epoch = 154 train_loss : 57.896088 , test loss : 52.823601
epoch = 155 train_loss : 57.765366 , test loss : 52.701195
epoch = 156 train_loss : 57.627930 , test loss : 52.567177
epoch = 157 train_loss : 57.258923 , test loss : 52.236595
epoch = 158 train_loss : 57.063034 , test loss : 52.029739
epoch = 160 train_loss : 56.713505 , test loss : 51.714340
epoch = 161 train_loss : 56.550419 , test loss : 51.585358
epoch = 162 train_loss : 56.344017 , test loss : 51.347023
epoch = 163 train_loss : 56.203003 , test loss : 51.241497
epoch = 164 train_loss : 55.990208 , test loss : 51.014557
epoch = 165 train_loss : 55.882462 , test loss : 50.916065
epoch = 166 train_loss : 55.645386 , test loss : 50.672001
epoch = 167 train_loss : 55.465515 , test loss : 50.508686
epoch = 169 train_loss : 55.153076 , test loss : 50.194958
epoch = 170 train_loss : 54.980690 , test loss : 50.051109
epoch = 171 train_loss : 54.810341 , test loss : 49.901672
epoch = 172 train_loss : 54.632904 , test loss : 49.6869

epoch = 360 train_loss : 38.758865 , test loss : 34.370003
epoch = 362 train_loss : 38.718620 , test loss : 34.329189
epoch = 367 train_loss : 38.554642 , test loss : 34.290344
epoch = 368 train_loss : 38.502808 , test loss : 34.209679
epoch = 369 train_loss : 38.466888 , test loss : 34.142532
epoch = 370 train_loss : 38.544647 , test loss : 34.124893
epoch = 374 train_loss : 38.326870 , test loss : 34.001205
epoch = 377 train_loss : 38.343723 , test loss : 33.930786
epoch = 378 train_loss : 38.280632 , test loss : 33.901962
epoch = 381 train_loss : 38.141670 , test loss : 33.873417
epoch = 382 train_loss : 38.161694 , test loss : 33.793835
epoch = 383 train_loss : 38.176174 , test loss : 33.756092
epoch = 387 train_loss : 38.038662 , test loss : 33.658596
epoch = 389 train_loss : 37.925968 , test loss : 33.607292
epoch = 392 train_loss : 37.840664 , test loss : 33.583981
epoch = 393 train_loss : 37.848736 , test loss : 33.490124
epoch = 395 train_loss : 37.777935 , test loss : 33.4673

epoch = 1053 train_loss : 33.906631 , test loss : 29.837877
epoch = 1105 train_loss : 33.858658 , test loss : 29.825220
epoch = 1109 train_loss : 33.825672 , test loss : 29.814447
epoch = 1114 train_loss : 33.879551 , test loss : 29.804144
epoch = 1129 train_loss : 33.852291 , test loss : 29.797880
epoch = 1138 train_loss : 33.803692 , test loss : 29.797783
epoch = 1148 train_loss : 33.808681 , test loss : 29.787867
epoch = 1160 train_loss : 33.797993 , test loss : 29.775429
epoch = 1166 train_loss : 33.788143 , test loss : 29.753630
epoch = 1207 train_loss : 33.749950 , test loss : 29.733604
epoch = 1224 train_loss : 33.772785 , test loss : 29.706675
epoch = 1256 train_loss : 33.734791 , test loss : 29.686760
epoch = 1315 train_loss : 33.775188 , test loss : 29.678291
epoch = 1354 train_loss : 33.750854 , test loss : 29.672173
epoch = 1376 train_loss : 33.668762 , test loss : 29.658360
epoch = 1390 train_loss : 33.677979 , test loss : 29.647678
epoch = 1418 train_loss : 33.658054 , te

In [61]:
n_folds=5

def rmlse_cv2(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x33)
    rmse=np.sqrt(-cross_val_score(model,x33,y33,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [62]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.005,random_state=1))
score=rmlse_cv2(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 5.6020 (0.5464)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


- use boxcox transformer to gaussian dist

In [63]:
from scipy.stats import boxcox,boxcox_normmax
from scipy.special import boxcox1p

In [64]:
all_train_data1.shape

(5760, 18)

In [65]:
all_train_data1.shape[1]

18

In [66]:
for i in range(all_train_data1.shape[1]):
    print(all_train_data1.index[i],'--skew : ',all_train_data1[i].skew(),'--kurt : ',all_train_data1[i].kurt())

0 --skew :  -0.2834134242907861 --kurt :  -0.7968009844193018
1 --skew :  -5.095255676523535 --kurt :  85.70760213466164
2 --skew :  9.165303973318487 --kurt :  132.25692190213024
3 --skew :  2.172048918138678 --kurt :  8.458548044518952
4 --skew :  5.378836580918865 --kurt :  41.25921290333162
5 --skew :  1.6391191931961593 --kurt :  3.8457481859587697
6 --skew :  2.071649558193799 --kurt :  6.419180514549189
7 --skew :  1.3121217461933794 --kurt :  5.2975502420733696
8 --skew :  1.103678593463204 --kurt :  1.7728459308764268
9 --skew :  1.2763236528649795 --kurt :  2.115391100082106
10 --skew :  21.7839204262489 --kurt :  623.8917732792679
11 --skew :  -0.6050425772235472 --kurt :  -0.1979575284081454
12 --skew :  2.1806637991938813 --kurt :  8.332926208370576
13 --skew :  -0.7973228533500623 --kurt :  21.5544747714386
14 --skew :  0.5569055078560654 --kurt :  -0.9777533530993172
15 --skew :  0.5166061431100419 --kurt :  -1.0455161114586577
16 --skew :  0.9251663396039612 --kurt :  1

In [67]:
all_train_data2=pd.DataFrame()
for i in range(all_train_data1.shape[1]):
    lm=boxcox_normmax(all_train_data1[i]+1)
    all_train_data2[i]=boxcox1p(all_train_data1[i],lm)
    print(all_train_data1.index[i],'--skew : ',all_train_data1[i].skew(),'--kurt : ',all_train_data1[i].kurt())
    print('lambda : ',lm)
    print(all_train_data2.index[i],'--skew : ',all_train_data2[i].skew(),'--kurt : ',all_train_data2[i].kurt())
    print('--------------------------------------------------------------------')

0 --skew :  -0.2834134242907861 --kurt :  -0.7968009844193018
lambda :  1.461645538288768
0 --skew :  -0.07414275326700835 --kurt :  -0.9258821506244836
--------------------------------------------------------------------
1 --skew :  -5.095255676523535 --kurt :  85.70760213466164
lambda :  5.990940784090655
1 --skew :  0.2052435896115669 --kurt :  0.583625236412292
--------------------------------------------------------------------
2 --skew :  9.165303973318487 --kurt :  132.25692190213024
lambda :  -3.259330902857795
2 --skew :  -0.003907562716963148 --kurt :  -0.06972890461562153
--------------------------------------------------------------------
3 --skew :  2.172048918138678 --kurt :  8.458548044518952
lambda :  -6.182620739009405
3 --skew :  0.05859262861967747 --kurt :  -0.5568735662256521
--------------------------------------------------------------------
4 --skew :  5.378836580918865 --kurt :  41.25921290333162
lambda :  -0.7826645126628972
4 --skew :  -0.03484059806052529 --

In [68]:
all_train_data2.shape

(5760, 18)

In [69]:
year_data2={}
for m in range(12):
    month_data2=np.empty((18,480))
    month_data2[:,:]=all_train_data2.T.iloc[:,m*480:(m+1)*480]
    year_data2[m]=month_data2
    

In [70]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data1.T.iloc[:,m*480:(m+1)*480]
    year_data3[m]=month_data3

In [71]:
year_data[11][-6,-5:]

array([2.6, 2.2, 2.3, 2.3, 2.3])

In [72]:
year_data3[11][-6,-5:]

array([2.6, 2.2, 2.3, 2.3, 2.3])

In [73]:
year_data2[11][-6,-5:]

array([1.1003593 , 1.01289662, 1.03598957, 1.03598957, 1.03598957])

In [74]:
x4=np.empty((12*471,18*9))
y4=np.empty((12*471,1))

for m in range(12):
    for g in range(471):
        x4[m*471+g:m*471+g+1,:]=year_data2[m][:,g:g+9].reshape(1,-1)
        y4[m*471+g:m*471+g+1,:]=year_data2[m][9,g+9]

In [75]:
x4[-6,81:90]

array([3.58579693, 3.58579693, 4.10611904, 2.28868315, 0.80300331,
       0.80300331, 3.29995821, 6.74890197, 8.4854749 ])

In [76]:
x4[-5,81:90]

array([3.58579693, 4.10611904, 2.28868315, 0.80300331, 0.80300331,
       3.29995821, 6.74890197, 8.4854749 , 9.79861041])

In [77]:
x4[-4,81:90]

array([4.10611904, 2.28868315, 0.80300331, 0.80300331, 3.29995821,
       6.74890197, 8.4854749 , 9.79861041, 8.4854749 ])

In [78]:
y4.shape

(5652, 1)

In [79]:
y4[-6:]

array([[9.79861041],
       [8.4854749 ],
       [7.33028592],
       [5.58250973],
       [6.74890197],
       [7.46817766]])

In [80]:
y4[-5:]

array([[8.4854749 ],
       [7.33028592],
       [5.58250973],
       [6.74890197],
       [7.46817766]])

In [81]:
y4[-4:]

array([[7.33028592],
       [5.58250973],
       [6.74890197],
       [7.46817766]])

In [82]:
x44=torch.Tensor(x4)
y44=torch.Tensor(y4)

In [84]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

epoch = 10 train_loss : 865.270142 , test loss : 876.853271
epoch = 11 train_loss : 729.240173 , test loss : 750.348816
epoch = 12 train_loss : 569.000305 , test loss : 587.707520
epoch = 13 train_loss : 474.301147 , test loss : 491.711670
epoch = 14 train_loss : 412.903687 , test loss : 421.863678
epoch = 15 train_loss : 328.753601 , test loss : 337.893707
epoch = 16 train_loss : 279.185181 , test loss : 288.253937
epoch = 17 train_loss : 242.764282 , test loss : 253.224182
epoch = 18 train_loss : 231.822372 , test loss : 243.374466
epoch = 19 train_loss : 201.548981 , test loss : 207.080368
epoch = 20 train_loss : 166.045059 , test loss : 173.408096
epoch = 21 train_loss : 160.892151 , test loss : 167.866394
epoch = 22 train_loss : 140.378372 , test loss : 146.100723
epoch = 23 train_loss : 118.020279 , test loss : 121.690590
epoch = 25 train_loss : 99.770233 , test loss : 103.701790
epoch = 26 train_loss : 85.717697 , test loss : 88.648453
epoch = 27 train_loss : 74.878693 , test lo

epoch = 112 train_loss : 3.050683 , test loss : 3.466446
epoch = 115 train_loss : 3.074229 , test loss : 3.422505
epoch = 120 train_loss : 2.969547 , test loss : 3.378531
epoch = 127 train_loss : 2.837395 , test loss : 3.195202
epoch = 135 train_loss : 2.787773 , test loss : 3.168627
epoch = 137 train_loss : 2.718089 , test loss : 3.095798
epoch = 139 train_loss : 2.702080 , test loss : 3.060643
epoch = 175 train_loss : 2.604833 , test loss : 2.949634
epoch = 179 train_loss : 2.499925 , test loss : 2.872345
epoch = 195 train_loss : 2.402539 , test loss : 2.740726
epoch = 221 train_loss : 2.280985 , test loss : 2.653148
epoch = 250 train_loss : 2.240645 , test loss : 2.565321
epoch = 285 train_loss : 2.069336 , test loss : 2.427116
epoch = 286 train_loss : 2.051176 , test loss : 2.408576
epoch = 287 train_loss : 2.030197 , test loss : 2.375095
epoch = 304 train_loss : 2.001273 , test loss : 2.353276
epoch = 323 train_loss : 2.016889 , test loss : 2.337834
epoch = 377 train_loss : 1.9501

epoch = 27 train_loss : 112.309906 , test loss : 103.760178
epoch = 28 train_loss : 102.590172 , test loss : 92.923210
epoch = 29 train_loss : 103.729691 , test loss : 91.707146
epoch = 30 train_loss : 87.756493 , test loss : 78.984154
epoch = 31 train_loss : 80.182785 , test loss : 73.845543
epoch = 32 train_loss : 73.211006 , test loss : 67.126640
epoch = 33 train_loss : 73.120255 , test loss : 64.905861
epoch = 34 train_loss : 61.721581 , test loss : 56.885422
epoch = 35 train_loss : 61.820541 , test loss : 54.825333
epoch = 36 train_loss : 51.203251 , test loss : 46.685135
epoch = 39 train_loss : 38.804169 , test loss : 35.075172
epoch = 40 train_loss : 36.483261 , test loss : 32.465714
epoch = 42 train_loss : 29.722006 , test loss : 26.654596
epoch = 44 train_loss : 24.129526 , test loss : 21.817984
epoch = 45 train_loss : 22.519215 , test loss : 20.088545
epoch = 46 train_loss : 20.386618 , test loss : 18.210249
epoch = 47 train_loss : 19.116007 , test loss : 17.807764
epoch = 49

epoch = 103 train_loss : 4.380663 , test loss : 4.314411
epoch = 104 train_loss : 4.331340 , test loss : 4.260428
epoch = 106 train_loss : 4.258894 , test loss : 4.167512
epoch = 110 train_loss : 4.087943 , test loss : 3.984759
epoch = 111 train_loss : 4.144804 , test loss : 3.918939
epoch = 112 train_loss : 4.033299 , test loss : 3.843167
epoch = 117 train_loss : 3.966663 , test loss : 3.753687
epoch = 118 train_loss : 3.836040 , test loss : 3.643212
epoch = 122 train_loss : 3.750094 , test loss : 3.567783
epoch = 123 train_loss : 3.648505 , test loss : 3.491772
epoch = 128 train_loss : 3.541286 , test loss : 3.353657
epoch = 134 train_loss : 3.520403 , test loss : 3.328753
epoch = 136 train_loss : 3.437516 , test loss : 3.283053
epoch = 149 train_loss : 3.278321 , test loss : 3.109665
epoch = 151 train_loss : 3.309325 , test loss : 3.057081
epoch = 152 train_loss : 3.150245 , test loss : 2.942414
epoch = 164 train_loss : 2.977495 , test loss : 2.798965
epoch = 187 train_loss : 3.0537

In [85]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],256),nn.Linear(256,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

epoch = 10 train_loss : 1027.714966 , test loss : 803.123779
epoch = 11 train_loss : 804.533569 , test loss : 637.075867
epoch = 12 train_loss : 572.939392 , test loss : 440.351593
epoch = 13 train_loss : 435.760376 , test loss : 333.020660
epoch = 14 train_loss : 346.166107 , test loss : 264.633606
epoch = 15 train_loss : 309.660278 , test loss : 247.608032
epoch = 16 train_loss : 203.545181 , test loss : 151.950653
epoch = 17 train_loss : 160.917343 , test loss : 119.423096
epoch = 18 train_loss : 129.467392 , test loss : 95.521271
epoch = 19 train_loss : 105.361618 , test loss : 77.656288
epoch = 20 train_loss : 86.619019 , test loss : 63.927036
epoch = 21 train_loss : 71.028069 , test loss : 52.553856
epoch = 22 train_loss : 60.910686 , test loss : 45.329247
epoch = 23 train_loss : 53.140537 , test loss : 39.980755
epoch = 24 train_loss : 39.956997 , test loss : 29.599483
epoch = 26 train_loss : 28.147449 , test loss : 21.824139
epoch = 27 train_loss : 23.151121 , test loss : 17.66

epoch = 917 train_loss : 1.764634 , test loss : 1.991262
epoch = 928 train_loss : 1.741005 , test loss : 1.943453
epoch = 955 train_loss : 1.743247 , test loss : 1.922879
--------------------------------------------------------------------------
fold 2,train loss mean : 1.743247,test loss : 1.922879
-------------------------------------------------------------------------
epoch = 5 train_loss : 789.126648 , test loss : 984.527100
epoch = 6 train_loss : 457.533691 , test loss : 569.942139
epoch = 7 train_loss : 277.393494 , test loss : 327.294830
epoch = 8 train_loss : 189.689331 , test loss : 213.810242
epoch = 9 train_loss : 144.255539 , test loss : 159.604889
epoch = 10 train_loss : 111.876884 , test loss : 120.172333
epoch = 11 train_loss : 88.204361 , test loss : 89.161430
epoch = 12 train_loss : 75.361389 , test loss : 74.186028
epoch = 13 train_loss : 67.428810 , test loss : 69.373520
epoch = 15 train_loss : 51.631096 , test loss : 50.710079
epoch = 16 train_loss : 46.239101 , te

epoch = 33 train_loss : 5.357525 , test loss : 4.876404
epoch = 35 train_loss : 4.716361 , test loss : 4.864124
epoch = 36 train_loss : 4.673799 , test loss : 4.770055
epoch = 42 train_loss : 4.855512 , test loss : 4.480691
epoch = 45 train_loss : 4.265597 , test loss : 4.436914
epoch = 48 train_loss : 4.139806 , test loss : 4.374228
epoch = 51 train_loss : 4.022134 , test loss : 4.324131
epoch = 54 train_loss : 4.092227 , test loss : 4.145712
epoch = 65 train_loss : 3.971111 , test loss : 3.869249
epoch = 83 train_loss : 3.700142 , test loss : 3.721615
epoch = 86 train_loss : 3.858247 , test loss : 3.695002
epoch = 104 train_loss : 3.243176 , test loss : 3.447855
epoch = 114 train_loss : 2.858245 , test loss : 3.349945
epoch = 176 train_loss : 2.927483 , test loss : 3.059493
epoch = 190 train_loss : 2.591629 , test loss : 2.689015
epoch = 287 train_loss : 2.898034 , test loss : 2.684316
epoch = 296 train_loss : 2.747541 , test loss : 2.597120
epoch = 299 train_loss : 2.260300 , test l

In [86]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],16),nn.Linear(16,1))
net1=get_net()
train_kfold(net1,1000,0.0005,5,x44,y44,256,0,0)

epoch = 42 train_loss : 1276.234863 , test loss : 982.818176
epoch = 43 train_loss : 1217.681274 , test loss : 919.159790
epoch = 44 train_loss : 1163.047241 , test loss : 885.548401
epoch = 45 train_loss : 1117.089233 , test loss : 845.618408
epoch = 46 train_loss : 1068.287598 , test loss : 812.427795
epoch = 47 train_loss : 1025.166992 , test loss : 781.474854
epoch = 48 train_loss : 984.612549 , test loss : 748.216797
epoch = 49 train_loss : 946.506958 , test loss : 717.506775
epoch = 50 train_loss : 905.436340 , test loss : 694.038696
epoch = 51 train_loss : 873.965820 , test loss : 676.828430
epoch = 52 train_loss : 834.967590 , test loss : 638.654236
epoch = 53 train_loss : 803.725037 , test loss : 612.728394
epoch = 54 train_loss : 773.353455 , test loss : 596.789612
epoch = 55 train_loss : 744.422363 , test loss : 565.911133
epoch = 56 train_loss : 716.021423 , test loss : 553.515869
epoch = 57 train_loss : 687.727966 , test loss : 523.956055
epoch = 58 train_loss : 661.243958

epoch = 204 train_loss : 7.355801 , test loss : 6.799399
epoch = 206 train_loss : 6.952429 , test loss : 6.621805
epoch = 207 train_loss : 6.797262 , test loss : 6.465499
epoch = 211 train_loss : 6.224964 , test loss : 6.168449
epoch = 213 train_loss : 6.313124 , test loss : 5.900477
epoch = 215 train_loss : 5.961926 , test loss : 5.662622
epoch = 219 train_loss : 5.310782 , test loss : 5.486964
epoch = 220 train_loss : 5.213592 , test loss : 5.414521
epoch = 221 train_loss : 5.553897 , test loss : 5.321567
epoch = 222 train_loss : 5.020434 , test loss : 5.174276
epoch = 223 train_loss : 4.938865 , test loss : 5.103993
epoch = 224 train_loss : 4.858054 , test loss : 5.033353
epoch = 225 train_loss : 4.805850 , test loss : 4.945472
epoch = 230 train_loss : 4.440490 , test loss : 4.717551
epoch = 231 train_loss : 4.383513 , test loss : 4.664765
epoch = 233 train_loss : 4.420022 , test loss : 4.525425
epoch = 236 train_loss : 4.516019 , test loss : 4.511156
epoch = 238 train_loss : 4.0365

epoch = 95 train_loss : 21.846390 , test loss : 32.720493
epoch = 96 train_loss : 21.412626 , test loss : 31.508825
epoch = 97 train_loss : 20.727798 , test loss : 31.322050
epoch = 98 train_loss : 19.972853 , test loss : 30.097555
epoch = 100 train_loss : 19.275883 , test loss : 29.089125
epoch = 101 train_loss : 18.234636 , test loss : 27.430071
epoch = 102 train_loss : 17.659040 , test loss : 26.540260
epoch = 104 train_loss : 16.281799 , test loss : 24.278379
epoch = 105 train_loss : 15.724431 , test loss : 23.389713
epoch = 106 train_loss : 15.290936 , test loss : 22.199675
epoch = 107 train_loss : 14.960458 , test loss : 21.471317
epoch = 108 train_loss : 14.243203 , test loss : 21.083288
epoch = 109 train_loss : 14.034598 , test loss : 20.856001
epoch = 110 train_loss : 13.698026 , test loss : 20.346285
epoch = 111 train_loss : 13.006719 , test loss : 19.198664
epoch = 112 train_loss : 12.443109 , test loss : 18.134005
epoch = 113 train_loss : 12.849122 , test loss : 17.671305
e

epoch = 72 train_loss : 201.203583 , test loss : 240.058304
epoch = 73 train_loss : 194.109192 , test loss : 232.263550
epoch = 74 train_loss : 185.468887 , test loss : 219.910110
epoch = 75 train_loss : 176.996704 , test loss : 209.421249
epoch = 76 train_loss : 165.178848 , test loss : 196.061188
epoch = 77 train_loss : 158.574646 , test loss : 188.538803
epoch = 78 train_loss : 150.749924 , test loss : 178.941040
epoch = 79 train_loss : 142.868790 , test loss : 169.259689
epoch = 80 train_loss : 135.655792 , test loss : 159.946136
epoch = 81 train_loss : 129.239563 , test loss : 152.099670
epoch = 82 train_loss : 122.616150 , test loss : 144.427933
epoch = 83 train_loss : 120.595108 , test loss : 142.092224
epoch = 84 train_loss : 111.088821 , test loss : 130.358734
epoch = 85 train_loss : 107.464302 , test loss : 125.618362
epoch = 86 train_loss : 100.793983 , test loss : 117.817184
epoch = 87 train_loss : 96.481621 , test loss : 112.517670
epoch = 88 train_loss : 92.369522 , test 

epoch = 54 train_loss : 682.321350 , test loss : 691.255981
epoch = 55 train_loss : 681.515198 , test loss : 684.996033
epoch = 56 train_loss : 654.955627 , test loss : 661.298706
epoch = 57 train_loss : 632.731750 , test loss : 645.324463
epoch = 58 train_loss : 620.443604 , test loss : 634.689148
epoch = 59 train_loss : 602.769897 , test loss : 612.640442
epoch = 60 train_loss : 589.344543 , test loss : 597.408569
epoch = 61 train_loss : 573.845398 , test loss : 584.692627
epoch = 62 train_loss : 559.205444 , test loss : 569.529724
epoch = 63 train_loss : 548.016235 , test loss : 560.070374
epoch = 64 train_loss : 531.687561 , test loss : 541.068970
epoch = 65 train_loss : 518.639099 , test loss : 528.927673
epoch = 66 train_loss : 505.123840 , test loss : 514.859436
epoch = 67 train_loss : 501.190582 , test loss : 514.177185
epoch = 68 train_loss : 478.556091 , test loss : 487.245636
epoch = 69 train_loss : 478.720734 , test loss : 481.137054
epoch = 70 train_loss : 456.652161 , tes

epoch = 439 train_loss : 2.019842 , test loss : 1.490376
epoch = 485 train_loss : 2.002088 , test loss : 1.484782
epoch = 521 train_loss : 2.055828 , test loss : 1.472823
epoch = 587 train_loss : 1.778583 , test loss : 1.429249
epoch = 588 train_loss : 1.774040 , test loss : 1.355055
epoch = 628 train_loss : 1.716945 , test loss : 1.326409
epoch = 656 train_loss : 1.677912 , test loss : 1.309002
epoch = 675 train_loss : 1.658308 , test loss : 1.295162
epoch = 701 train_loss : 1.665465 , test loss : 1.289906
epoch = 731 train_loss : 1.632328 , test loss : 1.280100
epoch = 766 train_loss : 1.632822 , test loss : 1.268447
epoch = 778 train_loss : 1.566100 , test loss : 1.255746
epoch = 846 train_loss : 1.532295 , test loss : 1.248083
epoch = 916 train_loss : 1.509952 , test loss : 1.208687
--------------------------------------------------------------------------
fold 4,train loss mean : 1.509952,test loss : 1.208687
------------------------------------------------------------------------

epoch = 189 train_loss : 9.298140 , test loss : 8.727533
epoch = 190 train_loss : 8.975140 , test loss : 8.302952
epoch = 191 train_loss : 8.750216 , test loss : 8.270124
epoch = 192 train_loss : 8.488057 , test loss : 7.799531
epoch = 193 train_loss : 8.757860 , test loss : 7.614789
epoch = 196 train_loss : 7.665653 , test loss : 7.011187
epoch = 197 train_loss : 7.532336 , test loss : 6.833210
epoch = 198 train_loss : 7.167998 , test loss : 6.783344
epoch = 200 train_loss : 6.831966 , test loss : 6.411687
epoch = 201 train_loss : 6.678332 , test loss : 6.286660
epoch = 202 train_loss : 6.655123 , test loss : 6.092815
epoch = 203 train_loss : 6.302726 , test loss : 6.090662
epoch = 204 train_loss : 6.152992 , test loss : 6.033479
epoch = 205 train_loss : 6.002482 , test loss : 5.873388
epoch = 206 train_loss : 5.884858 , test loss : 5.866535
epoch = 207 train_loss : 5.778393 , test loss : 5.827320
epoch = 208 train_loss : 5.914269 , test loss : 5.425859
epoch = 209 train_loss : 5.6444

In [88]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],32),nn.Linear(32,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

epoch = 9 train_loss : 1087.399658 , test loss : 859.643738
epoch = 10 train_loss : 973.209290 , test loss : 764.573486
epoch = 11 train_loss : 852.578369 , test loss : 689.124146
epoch = 12 train_loss : 763.217346 , test loss : 672.307190
epoch = 13 train_loss : 678.535400 , test loss : 558.754761
epoch = 14 train_loss : 607.090393 , test loss : 502.870087
epoch = 15 train_loss : 547.821106 , test loss : 475.138000
epoch = 16 train_loss : 489.564758 , test loss : 409.683075
epoch = 17 train_loss : 459.100677 , test loss : 351.679199
epoch = 18 train_loss : 397.218109 , test loss : 338.368683
epoch = 19 train_loss : 355.541962 , test loss : 294.249786
epoch = 20 train_loss : 321.779266 , test loss : 254.338837
epoch = 21 train_loss : 289.879791 , test loss : 250.158432
epoch = 22 train_loss : 259.085388 , test loss : 221.338120
epoch = 23 train_loss : 230.416245 , test loss : 183.073044
epoch = 24 train_loss : 205.497543 , test loss : 164.413666
epoch = 25 train_loss : 186.353821 , tes

epoch = 51 train_loss : 91.432617 , test loss : 137.444168
epoch = 52 train_loss : 85.097221 , test loss : 127.590576
epoch = 53 train_loss : 77.572807 , test loss : 117.901878
epoch = 54 train_loss : 76.262566 , test loss : 114.689377
epoch = 55 train_loss : 69.064667 , test loss : 104.782921
epoch = 56 train_loss : 66.240479 , test loss : 99.683304
epoch = 57 train_loss : 62.524292 , test loss : 93.991844
epoch = 59 train_loss : 53.636253 , test loss : 81.537720
epoch = 60 train_loss : 51.443871 , test loss : 77.799179
epoch = 61 train_loss : 47.383057 , test loss : 71.864395
epoch = 62 train_loss : 46.340023 , test loss : 69.662933
epoch = 63 train_loss : 43.709782 , test loss : 65.588875
epoch = 64 train_loss : 39.473221 , test loss : 59.766518
epoch = 65 train_loss : 37.648285 , test loss : 56.791431
epoch = 66 train_loss : 36.370789 , test loss : 54.350994
epoch = 67 train_loss : 32.160294 , test loss : 48.739285
epoch = 68 train_loss : 31.831917 , test loss : 47.310825
epoch = 6

epoch = 69 train_loss : 32.377735 , test loss : 38.295948
epoch = 70 train_loss : 30.392477 , test loss : 35.909206
epoch = 71 train_loss : 28.466331 , test loss : 32.647583
epoch = 73 train_loss : 25.225092 , test loss : 29.938574
epoch = 75 train_loss : 23.890083 , test loss : 24.622786
epoch = 77 train_loss : 19.619785 , test loss : 21.989067
epoch = 78 train_loss : 18.550114 , test loss : 21.764206
epoch = 79 train_loss : 17.477386 , test loss : 20.358299
epoch = 80 train_loss : 16.433363 , test loss : 18.133835
epoch = 81 train_loss : 15.493110 , test loss : 17.180258
epoch = 82 train_loss : 14.642300 , test loss : 16.783257
epoch = 83 train_loss : 13.883045 , test loss : 15.056643
epoch = 85 train_loss : 12.401034 , test loss : 13.543994
epoch = 86 train_loss : 12.365557 , test loss : 12.344419
epoch = 88 train_loss : 10.648276 , test loss : 12.071465
epoch = 89 train_loss : 10.131948 , test loss : 10.871696
epoch = 90 train_loss : 9.666787 , test loss : 10.858728
epoch = 92 trai

epoch = 101 train_loss : 23.896461 , test loss : 22.949154
epoch = 103 train_loss : 21.721159 , test loss : 20.678665
epoch = 104 train_loss : 18.696001 , test loss : 17.643377
epoch = 105 train_loss : 17.872303 , test loss : 16.785339
epoch = 106 train_loss : 16.231457 , test loss : 15.114268
epoch = 107 train_loss : 15.535627 , test loss : 14.371756
epoch = 108 train_loss : 14.462206 , test loss : 13.327853
epoch = 109 train_loss : 14.496446 , test loss : 13.280996
epoch = 110 train_loss : 12.578329 , test loss : 11.445168
epoch = 111 train_loss : 11.924495 , test loss : 10.694680
epoch = 113 train_loss : 10.178986 , test loss : 9.002866
epoch = 114 train_loss : 9.638524 , test loss : 8.409852
epoch = 115 train_loss : 9.342142 , test loss : 8.159196
epoch = 117 train_loss : 8.461914 , test loss : 7.307837
epoch = 118 train_loss : 8.105145 , test loss : 6.961794
epoch = 119 train_loss : 7.220909 , test loss : 5.945128
epoch = 120 train_loss : 6.958239 , test loss : 5.670778
epoch = 12

epoch = 408 train_loss : 1.846929 , test loss : 2.037919
epoch = 446 train_loss : 1.899713 , test loss : 2.019336
epoch = 484 train_loss : 1.798458 , test loss : 1.959599
epoch = 526 train_loss : 1.679294 , test loss : 1.934331
epoch = 530 train_loss : 1.668190 , test loss : 1.885556
epoch = 533 train_loss : 1.627133 , test loss : 1.882662
epoch = 539 train_loss : 1.687792 , test loss : 1.875335
epoch = 599 train_loss : 1.580560 , test loss : 1.852956
epoch = 635 train_loss : 1.600497 , test loss : 1.831594
epoch = 639 train_loss : 1.612547 , test loss : 1.824235
epoch = 715 train_loss : 1.541806 , test loss : 1.801443
epoch = 744 train_loss : 1.611763 , test loss : 1.772238
epoch = 790 train_loss : 1.610353 , test loss : 1.755741
epoch = 814 train_loss : 1.546196 , test loss : 1.737848
epoch = 968 train_loss : 1.536767 , test loss : 1.698238
--------------------------------------------------------------------------
fold 5,train loss mean : 1.536767,test loss : 1.698238
---------------

#### set lr = 0.0001 , batch_size = 64 , test loss <=1

In [95]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],16),nn.ReLU(),nn.Linear(16,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x44,y44,64,0,0)

epoch = 7 train_loss : 836.645447 , test loss : 888.281372
epoch = 8 train_loss : 732.458618 , test loss : 787.073547
epoch = 9 train_loss : 651.896240 , test loss : 714.205505
epoch = 10 train_loss : 567.893616 , test loss : 622.633850
epoch = 11 train_loss : 483.049316 , test loss : 528.091736
epoch = 12 train_loss : 411.271088 , test loss : 447.994904
epoch = 13 train_loss : 365.409210 , test loss : 394.608826
epoch = 14 train_loss : 328.151001 , test loss : 353.126190
epoch = 15 train_loss : 305.313354 , test loss : 328.451782
epoch = 16 train_loss : 268.280151 , test loss : 290.200439
epoch = 17 train_loss : 247.683899 , test loss : 269.292938
epoch = 18 train_loss : 227.256714 , test loss : 253.215576
epoch = 19 train_loss : 209.535110 , test loss : 232.961441
epoch = 20 train_loss : 202.986969 , test loss : 223.059097
epoch = 21 train_loss : 202.350098 , test loss : 223.007706
epoch = 22 train_loss : 168.298096 , test loss : 187.307037
epoch = 23 train_loss : 160.353195 , test l

epoch = 44 train_loss : 849.687988 , test loss : 995.237305
epoch = 45 train_loss : 808.278564 , test loss : 948.531189
epoch = 46 train_loss : 773.109497 , test loss : 908.943665
epoch = 47 train_loss : 742.144775 , test loss : 876.592285
epoch = 48 train_loss : 705.958557 , test loss : 835.634827
epoch = 49 train_loss : 675.384827 , test loss : 801.603210
epoch = 50 train_loss : 643.572815 , test loss : 765.335938
epoch = 51 train_loss : 615.217102 , test loss : 734.617920
epoch = 52 train_loss : 587.036316 , test loss : 702.545410
epoch = 53 train_loss : 560.943909 , test loss : 671.004089
epoch = 54 train_loss : 534.823486 , test loss : 641.886230
epoch = 55 train_loss : 510.487366 , test loss : 613.469482
epoch = 56 train_loss : 486.367889 , test loss : 585.809143
epoch = 57 train_loss : 466.027191 , test loss : 564.609619
epoch = 58 train_loss : 441.297882 , test loss : 534.164429
epoch = 59 train_loss : 420.467773 , test loss : 511.354614
epoch = 60 train_loss : 401.831055 , tes

epoch = 248 train_loss : 2.005594 , test loss : 4.186645
epoch = 255 train_loss : 1.902074 , test loss : 4.074046
epoch = 263 train_loss : 1.885746 , test loss : 4.060868
epoch = 269 train_loss : 1.856998 , test loss : 4.027891
epoch = 276 train_loss : 1.810866 , test loss : 3.951742
epoch = 279 train_loss : 1.795432 , test loss : 3.923733
epoch = 282 train_loss : 1.781962 , test loss : 3.908753
epoch = 287 train_loss : 1.761846 , test loss : 3.873209
epoch = 296 train_loss : 1.743356 , test loss : 3.848171
epoch = 300 train_loss : 1.754793 , test loss : 3.831624
epoch = 302 train_loss : 1.742598 , test loss : 3.808431
epoch = 304 train_loss : 1.695747 , test loss : 3.782326
epoch = 313 train_loss : 1.698411 , test loss : 3.761715
epoch = 315 train_loss : 1.706700 , test loss : 3.743372
epoch = 317 train_loss : 1.667647 , test loss : 3.719066
epoch = 327 train_loss : 1.646022 , test loss : 3.692386
epoch = 333 train_loss : 1.610693 , test loss : 3.660033
epoch = 337 train_loss : 1.5940

epoch = 3105 train_loss : 1.032694 , test loss : 2.611016
epoch = 3127 train_loss : 1.029366 , test loss : 2.605224
epoch = 3175 train_loss : 1.028786 , test loss : 2.601979
epoch = 3214 train_loss : 1.027735 , test loss : 2.597208
epoch = 3269 train_loss : 1.034992 , test loss : 2.596486
epoch = 3306 train_loss : 1.027704 , test loss : 2.588096
epoch = 3337 train_loss : 1.031014 , test loss : 2.587909
epoch = 3354 train_loss : 1.027330 , test loss : 2.577797
epoch = 3471 train_loss : 1.025091 , test loss : 2.572220
epoch = 3520 train_loss : 1.025979 , test loss : 2.570403
epoch = 3529 train_loss : 1.024176 , test loss : 2.564174
epoch = 3632 train_loss : 1.022849 , test loss : 2.557710
epoch = 3661 train_loss : 1.024280 , test loss : 2.557530
epoch = 3702 train_loss : 1.022682 , test loss : 2.549795
epoch = 3783 train_loss : 1.024477 , test loss : 2.547344
epoch = 3825 train_loss : 1.023521 , test loss : 2.536411
epoch = 3904 train_loss : 1.020146 , test loss : 2.531050
epoch = 3965 t

epoch = 109 train_loss : 18.071798 , test loss : 18.690523
epoch = 110 train_loss : 17.648289 , test loss : 18.253944
epoch = 111 train_loss : 16.825212 , test loss : 17.837301
epoch = 112 train_loss : 15.956547 , test loss : 16.684406
epoch = 113 train_loss : 15.359985 , test loss : 16.355490
epoch = 114 train_loss : 15.376201 , test loss : 16.137365
epoch = 115 train_loss : 14.402588 , test loss : 15.295532
epoch = 116 train_loss : 14.066903 , test loss : 14.904527
epoch = 117 train_loss : 13.823009 , test loss : 14.763923
epoch = 118 train_loss : 13.372408 , test loss : 14.448420
epoch = 119 train_loss : 12.758787 , test loss : 13.686208
epoch = 120 train_loss : 12.840057 , test loss : 13.571079
epoch = 122 train_loss : 11.606697 , test loss : 12.432731
epoch = 123 train_loss : 11.253688 , test loss : 12.226269
epoch = 124 train_loss : 10.750970 , test loss : 11.712164
epoch = 126 train_loss : 10.050228 , test loss : 11.049462
epoch = 128 train_loss : 9.436892 , test loss : 10.60484

epoch = 3232 train_loss : 1.090620 , test loss : 1.812286
epoch = 3317 train_loss : 1.094922 , test loss : 1.812157
epoch = 3429 train_loss : 1.078705 , test loss : 1.806581
epoch = 3503 train_loss : 1.081699 , test loss : 1.805188
epoch = 3611 train_loss : 1.088944 , test loss : 1.799506
epoch = 3612 train_loss : 1.089727 , test loss : 1.798263
epoch = 3638 train_loss : 1.082793 , test loss : 1.788036
epoch = 4018 train_loss : 1.062479 , test loss : 1.779656
epoch = 4324 train_loss : 1.058771 , test loss : 1.778351
epoch = 4752 train_loss : 1.063334 , test loss : 1.768924
--------------------------------------------------------------------------
fold 3,train loss mean : 1.063334,test loss : 1.768924
-------------------------------------------------------------------------
epoch = 4 train_loss : 907.584473 , test loss : 891.728455
epoch = 5 train_loss : 692.468933 , test loss : 665.090271
epoch = 6 train_loss : 589.364502 , test loss : 567.677917
epoch = 7 train_loss : 507.087463 , tes

epoch = 30 train_loss : 138.506561 , test loss : 115.133545
epoch = 31 train_loss : 118.268997 , test loss : 97.906921
epoch = 32 train_loss : 103.231255 , test loss : 84.074760
epoch = 33 train_loss : 91.694061 , test loss : 72.727806
epoch = 34 train_loss : 81.254944 , test loss : 63.376930
epoch = 35 train_loss : 72.119583 , test loss : 54.815247
epoch = 36 train_loss : 68.273163 , test loss : 50.825188
epoch = 37 train_loss : 58.378391 , test loss : 42.914028
epoch = 38 train_loss : 51.916866 , test loss : 36.639351
epoch = 39 train_loss : 48.052601 , test loss : 34.393681
epoch = 40 train_loss : 43.253407 , test loss : 30.213354
epoch = 41 train_loss : 38.307201 , test loss : 25.810965
epoch = 42 train_loss : 35.732639 , test loss : 23.712156
epoch = 43 train_loss : 32.457516 , test loss : 21.386049
epoch = 44 train_loss : 29.318357 , test loss : 19.108156
epoch = 45 train_loss : 26.995598 , test loss : 17.577387
epoch = 46 train_loss : 25.233252 , test loss : 16.129028
epoch = 47

In [96]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],8),nn.ReLU(),nn.Linear(8,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x44,y44,64,0,0)

epoch = 31 train_loss : 925.686279 , test loss : 947.561157
epoch = 32 train_loss : 856.153503 , test loss : 878.134399
epoch = 33 train_loss : 789.942444 , test loss : 811.811768
epoch = 34 train_loss : 733.046997 , test loss : 755.938843
epoch = 35 train_loss : 680.081787 , test loss : 704.644653
epoch = 36 train_loss : 625.771912 , test loss : 649.213562
epoch = 37 train_loss : 584.096985 , test loss : 608.328308
epoch = 38 train_loss : 536.030518 , test loss : 558.688049
epoch = 39 train_loss : 497.108429 , test loss : 518.552673
epoch = 40 train_loss : 458.582550 , test loss : 480.065613
epoch = 41 train_loss : 422.249237 , test loss : 443.441650
epoch = 42 train_loss : 409.615082 , test loss : 431.242859
epoch = 43 train_loss : 360.171356 , test loss : 381.793762
epoch = 44 train_loss : 331.814636 , test loss : 352.550629
epoch = 45 train_loss : 306.423157 , test loss : 327.218048
epoch = 46 train_loss : 275.691467 , test loss : 295.708801
epoch = 47 train_loss : 263.107117 , tes

epoch = 479 train_loss : 1.360233 , test loss : 1.467820
epoch = 489 train_loss : 1.362091 , test loss : 1.465003
epoch = 503 train_loss : 1.342997 , test loss : 1.446190
epoch = 517 train_loss : 1.335408 , test loss : 1.442152
epoch = 527 train_loss : 1.335292 , test loss : 1.440583
epoch = 539 train_loss : 1.327207 , test loss : 1.423561
epoch = 554 train_loss : 1.313745 , test loss : 1.411982
epoch = 564 train_loss : 1.300121 , test loss : 1.409226
epoch = 586 train_loss : 1.285073 , test loss : 1.392710
epoch = 588 train_loss : 1.291326 , test loss : 1.389034
epoch = 602 train_loss : 1.277387 , test loss : 1.379027
epoch = 613 train_loss : 1.271516 , test loss : 1.373594
epoch = 620 train_loss : 1.266951 , test loss : 1.369101
epoch = 654 train_loss : 1.268718 , test loss : 1.368785
epoch = 658 train_loss : 1.262086 , test loss : 1.358498
epoch = 662 train_loss : 1.246932 , test loss : 1.351161
epoch = 670 train_loss : 1.246554 , test loss : 1.349378
epoch = 678 train_loss : 1.2400

epoch = 132 train_loss : 6.886799 , test loss : 7.785685
epoch = 133 train_loss : 6.346088 , test loss : 7.072775
epoch = 134 train_loss : 5.963147 , test loss : 6.677495
epoch = 135 train_loss : 5.739305 , test loss : 6.412998
epoch = 138 train_loss : 5.434785 , test loss : 5.887299
epoch = 139 train_loss : 4.818614 , test loss : 5.274432
epoch = 142 train_loss : 4.306594 , test loss : 4.673787
epoch = 144 train_loss : 4.057559 , test loss : 4.380052
epoch = 145 train_loss : 3.988777 , test loss : 4.323388
epoch = 147 train_loss : 3.992572 , test loss : 4.215232
epoch = 148 train_loss : 3.631478 , test loss : 3.892445
epoch = 149 train_loss : 3.557856 , test loss : 3.814856
epoch = 151 train_loss : 3.393111 , test loss : 3.651299
epoch = 153 train_loss : 3.271529 , test loss : 3.524925
epoch = 158 train_loss : 3.077384 , test loss : 3.283967
epoch = 159 train_loss : 3.007279 , test loss : 3.260341
epoch = 161 train_loss : 2.939492 , test loss : 3.156980
epoch = 164 train_loss : 2.8583

epoch = 1726 train_loss : 1.089988 , test loss : 1.328499
epoch = 1787 train_loss : 1.085207 , test loss : 1.325596
epoch = 1799 train_loss : 1.084310 , test loss : 1.323475
epoch = 1816 train_loss : 1.082833 , test loss : 1.319723
epoch = 1862 train_loss : 1.080904 , test loss : 1.317783
epoch = 1928 train_loss : 1.076556 , test loss : 1.311335
epoch = 1931 train_loss : 1.078775 , test loss : 1.309566
epoch = 1964 train_loss : 1.074105 , test loss : 1.303404
epoch = 2069 train_loss : 1.069537 , test loss : 1.300682
epoch = 2085 train_loss : 1.069694 , test loss : 1.300490
epoch = 2104 train_loss : 1.070436 , test loss : 1.298431
epoch = 2106 train_loss : 1.067748 , test loss : 1.294081
epoch = 2114 train_loss : 1.066986 , test loss : 1.293967
epoch = 2124 train_loss : 1.068524 , test loss : 1.292974
epoch = 2158 train_loss : 1.064154 , test loss : 1.291520
epoch = 2172 train_loss : 1.064423 , test loss : 1.289030
epoch = 2240 train_loss : 1.068996 , test loss : 1.288926
epoch = 2261 t

epoch = 128 train_loss : 51.566200 , test loss : 55.174427
epoch = 129 train_loss : 48.913601 , test loss : 52.539665
epoch = 130 train_loss : 47.120010 , test loss : 50.871735
epoch = 131 train_loss : 44.879055 , test loss : 48.613586
epoch = 132 train_loss : 40.999931 , test loss : 44.485203
epoch = 133 train_loss : 40.443977 , test loss : 43.982796
epoch = 134 train_loss : 37.909729 , test loss : 41.313892
epoch = 135 train_loss : 34.010487 , test loss : 37.424580
epoch = 137 train_loss : 29.890339 , test loss : 33.441555
epoch = 138 train_loss : 28.345390 , test loss : 31.814341
epoch = 139 train_loss : 27.322578 , test loss : 30.755350
epoch = 140 train_loss : 24.955990 , test loss : 28.434408
epoch = 141 train_loss : 23.534407 , test loss : 27.021933
epoch = 142 train_loss : 21.956390 , test loss : 25.283154
epoch = 144 train_loss : 19.051769 , test loss : 22.450003
epoch = 146 train_loss : 16.795845 , test loss : 20.123489
epoch = 147 train_loss : 15.729167 , test loss : 19.1094

...

epoch = 658 train_loss : 2.318735 , test loss : 2.352938
epoch = 659 train_loss : 2.316435 , test loss : 2.350287
epoch = 664 train_loss : 2.307128 , test loss : 2.341735
epoch = 667 train_loss : 2.301053 , test loss : 2.336121
epoch = 669 train_loss : 2.296846 , test loss : 2.331681
epoch = 671 train_loss : 2.294274 , test loss : 2.330043
epoch = 672 train_loss : 2.292053 , test loss : 2.326405
epoch = 673 train_loss : 2.289809 , test loss : 2.325890
epoch = 676 train_loss : 2.288004 , test loss : 2.324553
epoch = 677 train_loss : 2.282256 , test loss : 2.318603
epoch = 684 train_loss : 2.269495 , test loss : 2.304689
epoch = 685 train_loss : 2.267235 , test loss : 2.303159
epoch = 691 train_loss : 2.257050 , test loss : 2.293927
epoch = 692 train_loss : 2.255291 , test loss : 2.290707
epoch = 696 train_loss : 2.249986 , test loss : 2.285162
epoch = 697 train_loss : 2.245918 , test loss : 2.281055
epoch = 702 train_loss : 2.243707 , test loss : 2.278524
epoch = 708 train_loss : 2.2301

epoch = 1215 train_loss : 1.658212 , test loss : 1.678098
epoch = 1221 train_loss : 1.655759 , test loss : 1.677773
epoch = 1227 train_loss : 1.653248 , test loss : 1.675873
epoch = 1228 train_loss : 1.654644 , test loss : 1.673643
epoch = 1235 train_loss : 1.654224 , test loss : 1.671911
epoch = 1236 train_loss : 1.644382 , test loss : 1.664298
epoch = 1245 train_loss : 1.640432 , test loss : 1.660599
epoch = 1259 train_loss : 1.629628 , test loss : 1.647010
epoch = 1274 train_loss : 1.621001 , test loss : 1.635217
epoch = 1288 train_loss : 1.619797 , test loss : 1.632892
epoch = 1289 train_loss : 1.609274 , test loss : 1.627167
epoch = 1292 train_loss : 1.608617 , test loss : 1.626397
epoch = 1298 train_loss : 1.607006 , test loss : 1.622767
epoch = 1300 train_loss : 1.604053 , test loss : 1.621464
epoch = 1305 train_loss : 1.600098 , test loss : 1.619053
epoch = 1312 train_loss : 1.595513 , test loss : 1.612404
epoch = 1319 train_loss : 1.594433 , test loss : 1.607199
epoch = 1323 t

epoch = 2369 train_loss : 1.262837 , test loss : 1.261389
epoch = 2377 train_loss : 1.260317 , test loss : 1.261118
epoch = 2388 train_loss : 1.258592 , test loss : 1.258508
epoch = 2390 train_loss : 1.260289 , test loss : 1.257139
epoch = 2400 train_loss : 1.261522 , test loss : 1.256901
epoch = 2414 train_loss : 1.256223 , test loss : 1.256820
epoch = 2428 train_loss : 1.254953 , test loss : 1.256523
epoch = 2430 train_loss : 1.254259 , test loss : 1.252658
epoch = 2441 train_loss : 1.255673 , test loss : 1.251001
epoch = 2464 train_loss : 1.252818 , test loss : 1.248238
epoch = 2465 train_loss : 1.248669 , test loss : 1.247805
epoch = 2476 train_loss : 1.248709 , test loss : 1.246935
epoch = 2484 train_loss : 1.247489 , test loss : 1.242773
epoch = 2493 train_loss : 1.245839 , test loss : 1.242053
epoch = 2498 train_loss : 1.243645 , test loss : 1.241967
epoch = 2507 train_loss : 1.243163 , test loss : 1.241409
epoch = 2520 train_loss : 1.243299 , test loss : 1.241262
epoch = 2534 t

epoch = 33 train_loss : 504.077637 , test loss : 495.011871
epoch = 34 train_loss : 472.789032 , test loss : 462.528900
epoch = 35 train_loss : 452.093475 , test loss : 444.130798
epoch = 36 train_loss : 429.994751 , test loss : 422.760040
epoch = 37 train_loss : 414.473206 , test loss : 406.133301
epoch = 38 train_loss : 392.981171 , test loss : 388.407074
epoch = 39 train_loss : 382.263824 , test loss : 379.380371
epoch = 40 train_loss : 354.606445 , test loss : 349.651398
epoch = 41 train_loss : 341.463104 , test loss : 336.816315
epoch = 42 train_loss : 319.109436 , test loss : 316.249756
epoch = 43 train_loss : 307.702301 , test loss : 306.692627
epoch = 44 train_loss : 290.749542 , test loss : 288.248383
epoch = 45 train_loss : 275.088470 , test loss : 273.134583
epoch = 46 train_loss : 258.164917 , test loss : 257.879486
epoch = 47 train_loss : 244.536972 , test loss : 244.251404
epoch = 48 train_loss : 231.626099 , test loss : 231.638885
epoch = 49 train_loss : 217.428131 , tes

epoch = 375 train_loss : 1.579044 , test loss : 1.416570
epoch = 386 train_loss : 1.583221 , test loss : 1.413214
epoch = 389 train_loss : 1.555922 , test loss : 1.399258
epoch = 396 train_loss : 1.547004 , test loss : 1.395611
epoch = 400 train_loss : 1.540438 , test loss : 1.381733
epoch = 421 train_loss : 1.521223 , test loss : 1.365133
epoch = 432 train_loss : 1.502997 , test loss : 1.351679
epoch = 462 train_loss : 1.461962 , test loss : 1.327042
epoch = 470 train_loss : 1.472206 , test loss : 1.325067
epoch = 473 train_loss : 1.452928 , test loss : 1.324358
epoch = 483 train_loss : 1.448999 , test loss : 1.321739
epoch = 495 train_loss : 1.436979 , test loss : 1.320289
epoch = 504 train_loss : 1.424888 , test loss : 1.285333
epoch = 522 train_loss : 1.406803 , test loss : 1.280250
epoch = 539 train_loss : 1.399077 , test loss : 1.272501
epoch = 560 train_loss : 1.402951 , test loss : 1.270554
epoch = 568 train_loss : 1.372872 , test loss : 1.257399
epoch = 576 train_loss : 1.3718

### linear function  test

In [83]:
def rmsle_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x4)
    rmse=np.sqrt(-cross_val_score(model,x4,y4,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [84]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
score=rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



Lasso score: 1.0524 (0.0607)



In [85]:
enet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9))
scoreo=rmsle_cv(enet)
print("\n enet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



 enet score: 1.0524 (0.0607)



In [86]:
ridge=make_pipeline(RobustScaler(),Ridge(alpha=0.5))
score=rmsle_cv(ridge)
print("\n ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


 ridge score: 1.0560 (0.0611)



In [87]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Gradient Boosting score: 1.2047 (0.0770)



In [88]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[10:58:14] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:58:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:58:32] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [90]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

LGBM score: 1.1398 (0.0725)



In [92]:
# x4_cxv=pd.DataFrame(x4.reshape(-1,9))
# x4_cxv.to_csv(r'F:\study\ml\HonyiLee2020\hw1\data\wx_x_cxv.csv')

In [93]:
x4[-5,81:91]

array([3.58579693, 4.10611904, 2.28868315, 0.80300331, 0.80300331,
       3.29995821, 6.74890197, 8.4854749 , 9.79861041, 0.        ])

In [94]:
x4[-4,81:91]

array([4.10611904, 2.28868315, 0.80300331, 0.80300331, 3.29995821,
       6.74890197, 8.4854749 , 9.79861041, 8.4854749 , 0.        ])

In [95]:
y4[-5:]

array([[8.4854749 ],
       [7.33028592],
       [5.58250973],
       [6.74890197],
       [7.46817766]])

In [96]:
x4_len=x4.shape[0]
print(int(0.8*x4_len))
x4_train=x4[:int(0.8*x4_len),:]
y4_train=y4[:int(0.8*x4_len),:]
x4_test=x4[int(0.8*x4_len):,:]
y4_test=y4[int(0.8*x4_len):,:]

4521


In [97]:
lasso=Lasso(alpha=0.0005,random_state=1)
lasso.fit(x4_train,y4_train)
y4_hat=lasso.predict(x4_test).reshape(-1,1)
np.sqrt(np.sum(np.power(y4_hat-y4_test,2),axis=0)/y4_test.shape[0])

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


array([1.08342463])

In [99]:
from scipy.special import inv_boxcox1p

In [100]:
y4_hat_orig=y4_hat
y4_test_orig=y4_test
y4_hat_orig=inv_boxcox1p(y4_hat_orig,0.4145678510523434)
y4_test_orig=inv_boxcox1p(y4_test_orig,0.4145678510523434)
np.sqrt(np.sum(np.power(y4_hat_orig-y4_test_orig,2),axis=0)/y4_test.shape[0])

array([5.54413321])